## TUTORIAL 01 - Thermal block problem
**_Keywords: certified reduced basis method, scalar problem_**

### 1. Introduction
In this Tutorial, we consider steady heat conduction in a two-dimensional domain $\Omega$.

<img src="data/thermal_block.png" />

We define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk centered at the origin of radius $r_0=0.5$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$. 

The conductivity $\kappa$ is assumed to be constant on $\Omega_1$ and $\Omega_2$, i.e.
$$
\kappa|_{\Omega_1}=\kappa_0 \quad \textrm{and} \quad \kappa|_{\Omega_2}=1.
$$

For this problem, we consider $P=2$ parameters:
1. the first one is related to the conductivity in $\Omega_1$, i.e. $\mu_0\equiv k_0$ (_note that parameters numbering is zero-based_);
2. the second parameter $\mu_1$ takes into account the constant heat flux over $\Gamma_{base}$.

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,10]\times[-1,1].
$$

In this problem we model the heat transfer process due to the heat flux over the bottom boundary $\Gamma_{base}$ and the following conditions on the remaining boundaries:
* the left and right boundaries $\Gamma_{side}$ are insulated,
* the top boundary $\Gamma_{top}$ is kept at a reference temperature (say, zero),

with the aim of measuring the average temperature on $\Gamma_{base}$.

In order to obtain a faster evaluation (yet, provably accurate) of the output of interest we propose to use a certified reduced basis approximation for the problem.

### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the temperature in the domain $\Omega$.

The strong formulation of the parametrized problem is given by:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that</center>

$$
\begin{cases}
	- \text{div} (\kappa(\mu_0)\nabla u(\boldsymbol{\mu})) = 0 & \text{in } \Omega,\\
	u(\boldsymbol{\mu}) = 0 & \text{on } \Gamma_{top},\\
	\kappa(\mu_0)\nabla u(\boldsymbol{\mu})\cdot \mathbf{n} = 0 & \text{on } \Gamma_{side},\\
	\kappa(\mu_0)\nabla u(\boldsymbol{\mu})\cdot \mathbf{n} = \mu_1 & \text{on } \Gamma_{base}.
\end{cases}
$$
<br>

where 
* $\mathbf{n}$ denotes the outer normal to the boundaries $\Gamma_{side}$ and $\Gamma_{base}$,
* the conductivity $\kappa(\mu_0)$ is defined as follows:
$$
\kappa(\mu_0) =
\begin{cases}
	\mu_0 & \text{in } \Omega_1,\\
	1 & \text{in } \Omega_2,\\
\end{cases}
$$

The corresponding weak formulation reads:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that</center>

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H^1(\Omega) : v|_{\Gamma_{top}}=0\}
$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \kappa(\mu_0)\nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \mu_1\int_{\Gamma_{base}}v \ ds.$$

The (compliant) output of interest $s(\boldsymbol{\mu})$ given by
$$s(\boldsymbol{\mu}) = \mu_1\int_{\Gamma_{base}} u(\boldsymbol{\mu})$$
is computed for each $\boldsymbol{\mu}$.

In [1]:
import sys
sys.path.append("../../MLniCS/")

from mlnics import NN, Losses, Normalization, RONNData
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

## 3. Affine decomposition

For this problem the affine decomposition is straightforward:
$$a(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega_1}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)} \ + \  \underbrace{1}_{\Theta^{a}_1(\boldsymbol{\mu})}\underbrace{\int_{\Omega_2}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_1(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{\mu_1}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Gamma_{base}}v \ ds}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class ThermalBlock(EllipticCoerciveCompliantProblem):
```
by specifying the coefficients $\Theta^{a}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):     
```
and the bilinear forms $a_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [2]:
class ThermalBlock(EllipticCoerciveCompliantProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        EllipticCoerciveCompliantProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)

    # Return custom problem name
    def name(self):
        return "ThermalBlock"

    # Return the alpha_lower bound.
    def get_stability_factor_lower_bound(self):
        return min(self.compute_theta("a"))

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = mu[0]
            theta_a1 = 1.
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = mu[1]
            return (theta_f0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            u = self.u
            a0 = inner(grad(u), grad(v)) * dx(1)
            a1 = inner(grad(u), grad(v)) * dx(2)
            return (a0, a1)
        elif term == "f":
            ds = self.ds
            f0 = v * ds(1)
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 3)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/thermal_block.xml")
subdomains = MeshFunction("size_t", mesh, "data/thermal_block_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/thermal_block_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the ThermalBlock class

In [5]:
problem = ThermalBlock(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a reduced basis method

In [6]:
reduction_method = PODGalerkin(problem)#ReducedBasis(problem)
reduction_method.set_Nmax(20)
reduction_method.set_tolerance(1e-10)#1e-5)

### 4.5. Perform the offline phase

In [7]:
reduction_method.initialize_training_set(100)
reduced_problem = reduction_method.offline()

### 4.5b Train the neural network

In [8]:
input_normalization = Normalization.StandardNormalization()
output_normalization = Normalization.IdentityNormalization()

In [9]:
pinn_net = NN.RONN(problem, reduction_method)
data = RONNData.RONNDataLoader(pinn_net)
_ = data.train_validation_split(0.2)

In [18]:
pinn_loss = Losses.PINN_Loss(pinn_net)
NN.normalize_and_train(pinn_net, data, pinn_loss, input_normalization, lr=0.0001, epochs=30000)

Getting operator matrices...
Getting operator matrices...
0 7.068675490132821e-06 	Loss(validation) = 0.016767848012642826
100 7.091320813345384e-06 	Loss(validation) = 0.00030383548826593096
200 6.9570355950844535e-06 	Loss(validation) = 0.0003039450139436846
300 6.949312187698776e-06 	Loss(validation) = 0.00030424152096918437
400 6.941671470796116e-06 	Loss(validation) = 0.00030441630968020613
500 6.932866556161893e-06 	Loss(validation) = 0.00030450742578739783
600 6.923091295724676e-06 	Loss(validation) = 0.0003045389142280752
700 6.912402059628829e-06 	Loss(validation) = 0.00030455484391917674
800 6.9007410440362345e-06 	Loss(validation) = 0.00030454695789095454
900 6.888331537242952e-06 	Loss(validation) = 0.0003045123224625666
1000 6.87462976846277e-06 	Loss(validation) = 0.00030444886865576903
1100 6.859950112125566e-06 	Loss(validation) = 0.00030437453989328866
1200 6.84433653475438e-06 	Loss(validation) = 0.00030426961681776887
1300 6.82719141948091e-06 	Loss(validation) = 0.0

11700 5.141404035552457e-06 	Loss(validation) = 0.0002820264686206028
11800 4.941440493990233e-06 	Loss(validation) = 0.0002867576256335802
11900 1.183719726548235e-05 	Loss(validation) = 0.00028759345446654794
12000 4.86823537799375e-06 	Loss(validation) = 0.0002848662990194594
12100 7.935724602648446e-06 	Loss(validation) = 0.00028125886342002455
12200 4.839399376826414e-06 	Loss(validation) = 0.00028449671783156374
12300 9.8233544888412e-06 	Loss(validation) = 0.00027452739400690474
12400 1.371333270569929e-05 	Loss(validation) = 0.00031697438340765147
12500 4.933544343407938e-06 	Loss(validation) = 0.0002791728175017405
12600 5.7102006948468124e-05 	Loss(validation) = 0.0002733759914947872
12700 4.769840956678431e-06 	Loss(validation) = 0.00028354673977106653
12800 5.354438558211975e-06 	Loss(validation) = 0.00028796758285543815
12900 5.091446942212526e-06 	Loss(validation) = 0.00028873822102935874
13000 4.762694103035244e-06 	Loss(validation) = 0.00028665144708973634
13100 1.74611

23400 0.00018676047037954137 	Loss(validation) = 0.0004297978949394025
23500 3.550167411709135e-06 	Loss(validation) = 0.000269074599380583
23600 1.5196346665097144e-05 	Loss(validation) = 0.00025767300405186354
23700 3.604451941937107e-06 	Loss(validation) = 0.000269660129389708
23800 4.444593551404239e-06 	Loss(validation) = 0.0002760618059037182
23900 3.993130271497036e-06 	Loss(validation) = 0.00027696224901924214
24000 3.529561468434992e-06 	Loss(validation) = 0.00026767737120408875
24100 3.604159298666238e-06 	Loss(validation) = 0.0002688737955666104
24200 6.096841900023971e-05 	Loss(validation) = 0.0002904662698852764
24300 3.4908571960278645e-06 	Loss(validation) = 0.0002686345162241961
24400 0.00012426930103010347 	Loss(validation) = 0.0004996504832430596
24500 3.4640467483134064e-06 	Loss(validation) = 0.0002675395096766684
24600 1.4140066603120006e-05 	Loss(validation) = 0.0002804323985420833
24700 3.444107225136031e-06 	Loss(validation) = 0.0002675697635234098
24800 8.12283

In [15]:
pdnn_net = NN.RONN(problem, reduction_method)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization)
NN.normalize_and_train(pdnn_net, data, pdnn_loss, input_normalization, lr=0.0001, epochs=30000)

0 0.28800297384091766 	Loss(validation) = 0.25529408622877886
100 0.013639913538200363 	Loss(validation) = 0.008296174888300983
200 0.009306583563641722 	Loss(validation) = 0.004940371502413011
300 0.0070491082336165915 	Loss(validation) = 0.0036989681949178405
400 0.005811029910248249 	Loss(validation) = 0.003173999306939377
500 0.00505051903615153 	Loss(validation) = 0.0029211272595938784
600 0.00449311628453082 	Loss(validation) = 0.0027467691112722913
700 0.004045567669623023 	Loss(validation) = 0.002605180020212612
800 0.0036701488693103414 	Loss(validation) = 0.002482572047749268
900 0.0033464901408495873 	Loss(validation) = 0.0023702114223137917
1000 0.003062223263694243 	Loss(validation) = 0.002262673699283172
1100 0.0028092190060756235 	Loss(validation) = 0.002157258076016971
1200 0.002581490312551859 	Loss(validation) = 0.0020525848362978063
1300 0.002373941773878686 	Loss(validation) = 0.001947428878834031
1400 0.0021815143742305227 	Loss(validation) = 0.0018398070822226473


11800 2.3591668938412933e-06 	Loss(validation) = 3.250075979521122e-05
11900 1.3585863587304684e-06 	Loss(validation) = 3.5238962050117905e-05
12000 1.336013600377139e-06 	Loss(validation) = 3.5028223251900965e-05
12100 1.3847044243727267e-06 	Loss(validation) = 3.6057634962420056e-05
12200 1.2960301365798229e-06 	Loss(validation) = 3.468502189350068e-05
12300 1.2544425131342388e-06 	Loss(validation) = 3.491188250182309e-05
12400 1.2368167829867877e-06 	Loss(validation) = 3.462619285882055e-05
12500 1.4455466061926156e-06 	Loss(validation) = 3.57751431320174e-05
12600 1.227014039724339e-06 	Loss(validation) = 3.370016432461935e-05
12700 1.9954594287700893e-06 	Loss(validation) = 3.5840434825036596e-05
12800 1.1764650568022719e-06 	Loss(validation) = 3.5571509563153624e-05
12900 1.1206976528964782e-06 	Loss(validation) = 3.472754195984279e-05
13000 1.1095151387519918e-06 	Loss(validation) = 3.471401418194016e-05
13100 1.0851982952278812e-06 	Loss(validation) = 3.473722455282816e-05
1320

23500 8.300422734150885e-07 	Loss(validation) = 4.072162700208382e-05
23600 3.2052408457266426e-07 	Loss(validation) = 3.7846095456513635e-05
23700 1.48822562796966e-06 	Loss(validation) = 3.487416907555979e-05
23800 3.5056626914200646e-07 	Loss(validation) = 3.718086023839122e-05
23900 3.0199879831245326e-07 	Loss(validation) = 3.741729782649272e-05
24000 3.310877848257046e-07 	Loss(validation) = 3.75226833874264e-05
24100 3.023777741231115e-07 	Loss(validation) = 3.753026212190709e-05
24200 2.942907175201677e-07 	Loss(validation) = 3.777515370762247e-05
24300 7.123467334918106e-07 	Loss(validation) = 3.799739832131497e-05
24400 2.8808135806839873e-07 	Loss(validation) = 3.767762327868251e-05
24500 2.871264521853932e-07 	Loss(validation) = 3.7633802124520766e-05
24600 2.873421012103042e-07 	Loss(validation) = 3.757901334435063e-05
24700 1.0121857805278024e-06 	Loss(validation) = 3.651273373278867e-05
24800 4.675395135241992e-07 	Loss(validation) = 3.933108911861898e-05
24900 2.8784418

In [16]:
prnn_net = NN.RONN(problem, reduction_method)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization, omega=10.)
NN.normalize_and_train(prnn_net, data, prnn_loss, input_normalization, lr=0.0001, epochs=30000)

Getting operator matrices...
Getting operator matrices...
0 5.8629369347697615 	Loss(validation) = 4.731958919593261
100 0.3360269784237965 	Loss(validation) = 0.24354375922566507
200 0.19516904340635677 	Loss(validation) = 0.13818870838783567
300 0.14825298568273476 	Loss(validation) = 0.09812959450884545
400 0.11829413634760332 	Loss(validation) = 0.07374720749887179
500 0.0991246190704028 	Loss(validation) = 0.05860798323883663
600 0.08599085121488959 	Loss(validation) = 0.04835929868747142
700 0.07627004418940982 	Loss(validation) = 0.041035142084016415
800 0.06877549722560275 	Loss(validation) = 0.035967586460229566
900 0.06293282953447354 	Loss(validation) = 0.032814405885821324
1000 0.05835142853444536 	Loss(validation) = 0.031168417552709406
1100 0.054702594916940024 	Loss(validation) = 0.03054396103149702
1200 0.05171288240457039 	Loss(validation) = 0.03047638893875493
1300 0.04917470981289718 	Loss(validation) = 0.030603220287002235
1400 0.04694440014149818 	Loss(validation) 

12100 0.00017317420319567568 	Loss(validation) = 0.0004928328670685306
12200 0.0001656376497306912 	Loss(validation) = 0.0004938682197340326
12300 0.00015841257917050596 	Loss(validation) = 0.0004910555696928913
12400 0.00017102673450347306 	Loss(validation) = 0.00046440261933707456
12500 0.00014476752211207934 	Loss(validation) = 0.0004899642089895797
12600 0.00013864423879693312 	Loss(validation) = 0.0004954236738675729
12700 0.0001325035306889992 	Loss(validation) = 0.0004923282615899671
12800 0.00012685959887803776 	Loss(validation) = 0.000502178717792233
12900 0.0001212603007701936 	Loss(validation) = 0.00048235237736146826
13000 0.00011602780640946192 	Loss(validation) = 0.0004963205487891261
13100 0.000111262517563661 	Loss(validation) = 0.0004948866825531881
13200 0.00010640157269332939 	Loss(validation) = 0.0004991089265681055
13300 0.00010284560642051852 	Loss(validation) = 0.00048780877929874294
13400 9.771348104839334e-05 	Loss(validation) = 0.0005026160816741455
13500 0.00

23800 3.946298066790475e-05 	Loss(validation) = 0.0004621595471743681
23900 1.235285722869025e-05 	Loss(validation) = 0.0004899833972262653
24000 0.00019357037523747546 	Loss(validation) = 0.0008015197130696913
24100 1.2114140389218286e-05 	Loss(validation) = 0.00048573479507702986
24200 1.4839906059014157e-05 	Loss(validation) = 0.0004711771952642328
24300 1.1873765784898966e-05 	Loss(validation) = 0.0004835926944799443
24400 3.306505566739239e-05 	Loss(validation) = 0.00046586855639983184
24500 1.164708384150154e-05 	Loss(validation) = 0.0004812689329720323
24600 1.3625788447362506e-05 	Loss(validation) = 0.00046615370994746764
24700 1.1455696618126388e-05 	Loss(validation) = 0.0004798509893952632
24800 0.00025864876954615184 	Loss(validation) = 0.0007634567233473964
24900 1.1212943760644992e-05 	Loss(validation) = 0.0004749239489517669
25000 1.2326045272337735e-05 	Loss(validation) = 0.00045850984842497715
25100 1.1039835697916766e-05 	Loss(validation) = 0.0004699117445749914
25200 

In [17]:
reduction_method.initialize_testing_set(100)
test_mu = torch.tensor(reduction_method.testing_set)

nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

_ = NN.error_analysis_by_network(nets, test_mu, input_normalization, euclidean=False)

Error analysis for pinn_net
Mean Relative Error:
N	NN-HF			NN-RO			RO-HF
4	0.012393654591884441	0.01239365352061906	5.180639436173207e-07

Error analysis for pdnn_net
Mean Relative Error:
N	NN-HF			NN-RO			RO-HF
4	0.015777065255014655	0.01577706840116211	5.180639436173207e-07

Error analysis for prnn_net
Mean Relative Error:
N	NN-HF			NN-RO			RO-HF
4	0.007419653001663673	0.007419652015496423	5.180639436173207e-07



### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(100)
reduction_method.error_analysis()

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.initialize_testing_set(100)
reduction_method.speedup_analysis()

## 5. Assignments
1. Assume now also the conductivity on $\Omega_2$ to be paramerized, i.e.
$$
\kappa(\mu_0, \mu_2) =
\begin{cases}
	\mu_0 & \text{in } \Omega_1,\\
	\mu_2 & \text{in } \Omega_2,\\
\end{cases}
$$
for
$$
\boldsymbol{\mu} = (\mu_0, \mu_1, \mu_2) \in \mathbb{P}=[0.1,10]\times[-1,1] \times [0.1,10].
$$
Create a copy of this notebook and update the code accordingly. _Suggestion: for every new notebook copy change the value returned by the name() method of the ThermalBlock class to avoid conflicts between this notebook and your copy_.

2. Create another copy of this notebook, and change the model reduction technique from certified reduced basis to POD-Galerkin. Compare the results of the error analysis and speedup analysis for the two reduction techniques.

3. [*] Why is the $H^1$ seminorm used on $\mathbb{V}$? What would you need to change by using the $H^1$ norm instead?